In [ ]:
import os
import json
import random
import gym
import numpy as np
from gym import spaces
from stable_baselines3 import PPO
from dotenv import load_dotenv
import openai

In [ ]:
# Load OpenAI API Key securely from .env file
def load_api_key():
    env_path = "../models/key.env"
    load_dotenv(env_path)  # Load API key from .env file
    api_key = os.getenv("OPENAI_API_KEY")

    if not api_key:
        raise ValueError("❌ OpenAI API Key not found. Make sure it's in the .env file.")
    
    openai.api_key = api_key
    print("✅ OpenAI API Key loaded successfully!")



In [ ]:
# Load the dataset
def load_dataset():
    """Loads the JSON dataset for emotion-based scenarios."""
    dataset_path = "../datasets/game.json"

    try:
        with open(dataset_path, "r") as file:
            dataset = json.load(file)
        print("✅ Dataset loaded successfully!")
        return dataset["intents"]
    except FileNotFoundError:
        print(f"❌ Error: Dataset file '{dataset_path}' not found.")
        return []
    except json.JSONDecodeError:
        print(f"❌ Error: Failed to parse JSON file '{dataset_path}'.")
        return []

In [ ]:
# Define the Emotion-Based RL Environment
class EmotionGameEnv(gym.Env):
    """Custom Gym environment for Emotion-Based RL."""
    
    def __init__(self):
        super(EmotionGameEnv, self).__init__()

        # Load emotion dataset
        self.dataset = load_dataset()

        # Define state and action spaces
        self.emotions = ["sad", "angry", "happy", "stress", "neutral", "fear"]
        self.action_space = spaces.Discrete(4)  # 4 choices per scenario
        self.observation_space = spaces.Discrete(len(self.emotions))

        self.state = None
        self.current_scenario = None

    def reset(self):
        """Resets the environment for a new episode."""
        self.state = random.choice(self.emotions)  # Pick a random emotion
        self.current_scenario = random.choice(self.dataset)  # Pick a random scenario
        return self.emotions.index(self.state)

    def step(self, action):
        """Takes an action, applies it to the environment, and gets a reward."""
        try:
            responses = self.current_scenario["responses"]
            reward = responses[action]["reward"]
        except (IndexError, KeyError):
            reward = 0  # Default reward if action index is invalid
        
        done = True  # One step per episode
        info = {"scenario": self.current_scenario["scenario"]}
        
        return self.emotions.index(self.state), reward, done, info


In [ ]:
def get_gpt_response(emotion, scenario):
    """Generates a GPT-based response for an emotional scenario."""
    prompt = f"The user is feeling {emotion}. The situation is: '{scenario}'. What should they do? Provide a short helpful response."

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Use gpt-4o-mini instead of gpt-4
            messages=[
                {"role": "system", "content": "You are an AI therapist helping users manage emotions."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:

# Train the RL Model
def train_rl_model(env, total_timesteps=10000):
    """Trains the RL model using Proximal Policy Optimization (PPO)."""
    model = PPO("MlpPolicy", env, verbose=1)

    try:
        print("\n🎯 Training PPO model...\n")
        model.learn(total_timesteps=total_timesteps)
        model.save("emotional_rl_agent")
        print("✅ Model training completed and saved as 'emotional_rl_agent'.")
    except KeyboardInterrupt:
        print("⏸ Training interrupted. Saving model...")
        model.save("emotional_rl_agent")
    
    return model


In [ ]:
# Load and Evaluate the Model
def evaluate_model(env, model_path="emotional_rl_agent", num_tests=10):
    """Loads a trained model and runs test cases."""
    print("\n📥 Loading trained model...")
    model = PPO.load(model_path)

    obs = env.reset()
    for i in range(num_tests):
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)

        # Get GPT-generated response
        gpt_feedback = get_gpt_response(env.state, info["scenario"])

        # Display results
        print(f"\n🎭 Emotion: {env.state}")
        print(f"📜 Scenario: {info['scenario']}")
        print(f"✅ Action Taken: {action}")
        print(f"🏆 Reward: {reward}")
        print(f"💬 AI Therapy Response: {gpt_feedback}\n")

        if done:
            obs = env.reset()


In [ ]:
# Main Execution Function
def main():
    """Main function to execute the complete RL training & GPT-based evaluation."""
    
    print("🚀 Starting Emotion-Based RL & AI Therapy System...\n")
    
    # Load OpenAI API Key
    load_api_key()

    # Initialize the environment
    env = EmotionGameEnv()

    # Train the RL model
    model = train_rl_model(env)

    # Evaluate the trained model
    evaluate_model(env)

# Run the Main Function
if __name__ == "__main__":
    main()


🚀 Starting Emotion-Based RL & AI Therapy System...

✅ OpenAI API Key loaded successfully!
✅ Dataset loaded successfully!
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\dell\anaconda3\envs\new_bert\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(



🎯 Training PPO model...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -0.01    |
| time/              |          |
|    fps             | 1120     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | 0.55        |
| time/                   |             |
|    fps                  | 777         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.026947517 |
|    clip_fraction        | 0.756       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.36       |
|    explained_variance   | -0.00612    |
|   

In [ ]:
import openai

try:
    models = openai.Model.list()
    available_models = [model["id"] for model in models["data"]]
    print("✅ Available models:", available_models)
except Exception as e:
    print("❌ Error:", str(e))


✅ Available models: ['dall-e-2', 'o1-mini-2024-09-12', 'o1-preview-2024-09-12', 'o1-mini', 'o1-preview', 'whisper-1', 'dall-e-3', 'babbage-002', 'omni-moderation-latest', 'omni-moderation-2024-09-26', 'tts-1-hd-1106', 'gpt-4o-mini', 'gpt-4o-mini-2024-07-18', 'tts-1-hd', 'tts-1', 'gpt-3.5-turbo-16k', 'tts-1-1106', 'davinci-002', 'gpt-3.5-turbo-1106', 'gpt-3.5-turbo-instruct', 'gpt-3.5-turbo-instruct-0914', 'gpt-3.5-turbo-0125', 'gpt-3.5-turbo', 'text-embedding-3-large', 'text-embedding-3-small', 'text-embedding-ada-002']
